In [1]:
%%capture
! pip install ml-collections
! pip3 install pickle5
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab Notebooks/WhenPigsFlyContext/baselines/DeepGaze

In [2]:
import sys
import cv2
import time
from matplotlib import pyplot as plt
from tqdm import tqdm, trange
import numpy as np
import pandas as pd
import pickle
import pickle5 as pickle5
import random
from random import sample
import copy

import os
import shutil
from PIL import Image, ImageDraw
from google.colab.patches import cv2_imshow

import torch
from torch.utils.data import Dataset
from torchvision.transforms.functional import to_tensor, normalize
from torchvision import transforms

import matplotlib.pyplot as plt
import numpy as np
from scipy.misc import face
from scipy.ndimage import zoom
from scipy.special import logsumexp
import torch
import deepgaze_pytorch
from deepgaze_pytorch import modules

from utils import *
sys.path.append("..")
from SCEGRAM.SCEGRAM import SCEGRAM

In [3]:
context_dir = "../../SCEGRAM/01scenes/01object_present"
target_dir = "../../SCEGRAM/02objects"
info_dir = "../../SCEGRAM/SCEGRAM_Database_scenes_objects.xlsx"
context_size, target_size = (224, 224), (224, 224)
dataset = SCEGRAM(info_dir, context_dir, target_dir, context_size, target_size, is_transform=False)

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [6]:
with open("../IVSN/[SCEGRAM]bin_idxs.pkl", "rb") as tf:
    bin_info = pickle5.load(tf) 

In [4]:
def logsearchProcess(x, y, tg_xy, attentionMap, image_size, size, coef):
    mask_size = size
    tg_x, tg_y, w, h = tg_xy
    tg_xmax, tg_ymax = tg_x + w, tg_y + h 

    attenNP = (attentionMap[0,:,:].detach() * coef[0,:,:].detach()).numpy()
    y_fix, x_fix = y, x

    x_max_s, x_min_s, y_max_s, y_min_s = min(x_fix+mask_size//2, image_size[1]-1), max(x_fix-mask_size//2, 0), min(y_fix+mask_size//2, image_size[0]-1), max(y_fix-mask_size//2, 0)

    if x_max_s < tg_x or x_min_s > tg_xmax or y_max_s < tg_y or y_min_s > tg_ymax:
        coef[0, y_min_s:y_max_s+1, x_min_s:x_max_s+1] = 1000
        attenNP = (attentionMap[0,:,:].detach() * coef[0,:,:].detach()).numpy()
        y_fix, x_fix = np.unravel_index(attenNP.argmax(), attenNP.shape)
        return False, [x_fix, y_fix], coef

    return True, [], coef

def fixation_initialize():
    k, x_range, y_range = 4, 50, 30
    x_init, y_init = 1680//2, 1050//2
    ratio_horizontal, ratio_vertical = 1680/512, 1050/320
    random_num_x, random_num_y = random.sample(list(range(x_range)), 4), random.sample(list(range(y_range)), 4)
    x, y = [], []
    for i in range(4):
        choice_x, choice_y = random.choice([0, 1]), random.choice([0, 1])
        x_cord = int((x_init+random_num_x[i])/ratio_horizontal) if choice_x == 0 else int((x_init-random_num_x[i])/ratio_horizontal)
        y_cord = int((y_init+random_num_y[i])/ratio_vertical) if choice_y == 0 else int((y_init-random_num_y[i])/ratio_vertical)
        x.append(x_cord)
        y.append(y_cord)

    return x, y

In [5]:
%%capture
DEVICE = 'cuda'
img_size = (320, 512)
# you can use DeepGazeI or DeepGazeIIE
model = deepgaze_pytorch.DeepGazeIII(pretrained=True).to(DEVICE)
image = face()
centerbias_template = np.load('centerbias_mit1003.npy')
# rescale to match image size
centerbias = zoom(centerbias_template, (image.shape[0]/centerbias_template.shape[0], image.shape[1]/centerbias_template.shape[1]), order=0, mode='nearest')
# renormalize log density
centerbias -= logsumexp(centerbias)
centerbias_tensor = torch.tensor([centerbias]).to(DEVICE)
centerbias_tensor = transforms.Resize(img_size)(centerbias_tensor)

In [10]:
size = 48
deepgaze_CON_0_25, deepgaze_CON_25_50 = [], []
deepgaze_INCON_0_25, deepgaze_INCON_25_50 = [], []
scanpath, deepgaze_attention_map = {}, {}
selected_imgs = bin_info['con_(0, 25]'].tolist() + bin_info['con_(25, 50]'].tolist() + bin_info['incon_(0, 25]'].tolist() + bin_info['incon_(25, 50]'].tolist()

for id in trange(0, len(dataset)):
    if id not in selected_imgs:
        continue

    img, _, bbox_relative, category = dataset[id]
    # get the target bounding box
    tg_loc = bbox_cordinates(bbox_relative, img_size[1], img_size[0])
    history_x, history_y = fixation_initialize()
    # transform img to tensor
    img_tensor = torch.tensor([img.transpose(2, 0, 1)]).to(DEVICE)
    img_tensor = transforms.Resize(img_size)(img_tensor)

    count, max_search, coef, path = 0, 999, torch.ones((1, 320, 512)), []
    while count < max_search:
        fixation_history_x = np.array(history_x)
        fixation_history_y = np.array(history_y)
        x_hist_tensor = torch.tensor([fixation_history_x[model.included_fixations]]).to(DEVICE)
        y_hist_tensor = torch.tensor([fixation_history_y[model.included_fixations]]).to(DEVICE)
        log_density_prediction = model(img_tensor, centerbias_tensor, x_hist_tensor, y_hist_tensor)
        
        path.append([history_x[-1], history_y[-1]])
        isTg, coordinates, coef = logsearchProcess(history_x[-1], history_y[-1], tg_loc, log_density_prediction.squeeze(0).cpu(), img_size, size, coef)
        count += 1

        if isTg:
            # deepgaze_attention_map[id] = log_density_prediction.squeeze(0).cpu()
            break

        history_x.append(coordinates[0])
        history_y.append(coordinates[1])

    scanpath[id] = path

    if id in bin_info['con_(0, 25]'].tolist():
        deepgaze_CON_0_25.append(count)
    elif id in bin_info['con_(25, 50]'].tolist():
        deepgaze_CON_25_50.append(count)

    elif id in bin_info['incon_(0, 25]'].tolist():
        deepgaze_INCON_0_25.append(count)
    elif id in bin_info['incon_(25, 50]'].tolist():
        deepgaze_INCON_25_50.append(count)

    print("search times_{}: ".format(id), count)

deepgaze_CON_res = deepgaze_CON_0_25 + deepgaze_CON_25_50
deepgaze_INCON_res = deepgaze_INCON_0_25 + deepgaze_INCON_25_50
deepgaze_res = deepgaze_CON_res + deepgaze_INCON_res

  0%|          | 1/372 [00:03<19:29,  3.15s/it]

search times_0:  13


  1%|          | 2/372 [00:04<12:38,  2.05s/it]

search times_1:  5


  1%|          | 3/372 [00:12<30:53,  5.02s/it]

...

search times_364:  60


 99%|█████████▉| 370/372 [23:25<00:06,  3.19s/it]

search times_369:  16


100%|██████████| 372/372 [23:31<00:00,  3.79s/it]

search times_371:  20


In [11]:
np.mean(deepgaze_res), np.mean(deepgaze_CON_res), np.mean(deepgaze_INCON_res)

(27.898395721925134, 18.3125, 29.87741935483871)

In [17]:
def sampleIncon(incon_bin_result, con_bin_result, times):
    sample_times = times
    nums = len(con_bin_result)
    res = np.array([0.0] * 25)
    
    for id in range(sample_times):
        temp = sample(incon_bin_result, nums)
        temp_accu = [0] + model_performance(temp, len(temp))
        res += np.array(temp_accu[:25])

    return (res/sample_times).tolist()

def balanced_accu(res_con, res_incon):
    res = []
    for i in range(25):
        res.append((res_con[i]+res_incon[i])/2)

    return res

In [18]:
times = 100
deepgaze_CON_accu = [0] + model_performance(deepgaze_CON_res, len(deepgaze_CON_res))
deepgaze_INCON_accu = sampleIncon(deepgaze_INCON_res, deepgaze_CON_res, times)
deepgaze_accu = balanced_accu(deepgaze_CON_accu, deepgaze_INCON_accu)
deepgaze_accu[:11], deepgaze_CON_accu[:11], deepgaze_INCON_accu[:11]

([0.0,
  0.14546875,
  0.22140625,
  0.271875,
  0.3215625,
  0.36875,
  0.38890625,
  0.42484374999999996,
  0.46046875,
  0.48125,
  0.5081249999999999],
 [0,
  0.1875,
  0.3125,
  0.375,
  0.40625,
  0.4375,
  0.46875,
  0.53125,
  0.5625,
  0.5625,
  0.59375],
 [0.0,
  0.1034375,
  0.1303125,
  0.16875,
  0.236875,
  0.3,
  0.3090625,
  0.3184375,
  0.3584375,
  0.4,
  0.4225])

In [19]:
deepgaze_SCEGRAM_res = {}
deepgaze_SCEGRAM_res['combined_accu'] = deepgaze_accu
deepgaze_SCEGRAM_res['con_accu'] = deepgaze_CON_accu
deepgaze_SCEGRAM_res['incon_accu'] = deepgaze_INCON_accu
deepgaze_SCEGRAM_res['con_[0,25)'] = deepgaze_CON_0_25
deepgaze_SCEGRAM_res['con_[25,50)'] = deepgaze_CON_25_50
deepgaze_SCEGRAM_res['incon_[0,25)'] = deepgaze_INCON_0_25
deepgaze_SCEGRAM_res['incon_[25,50)'] = deepgaze_INCON_25_50
deepgaze_SCEGRAM_res['scanpath'] = scanpath
deepgaze_SCEGRAM_res['attention_map'] = deepgaze_attention_map

In [20]:
with open("../results/SCEGRAM/SCEGRAM(invariant_bin1_2)_deepgaze_res.pkl", "wb") as tf:
    pickle.dump(deepgaze_SCEGRAM_res, tf)